In [ ]:
import numpy as np
import pandas as pd
import time
import random
from helper_functions import *

In [ ]:
#problem_instance = "inst_tuning\heur040_n_300_m_13358"
instance_type = "test_instances/" #"inst_competition/" #"inst_tuning/" # 
problem_instance = "heur002_n_100_m_3274" #"heur040_n_300_m_13358" #"heur001_n_10_m_31" 
path = "data/"+instance_type + problem_instance +".txt"

In [ ]:
nodes, edges, s, n, m = create_problem_instance(path)
is_splex(nodes = nodes, plex_number = 2, s=s)

# Construction heuristic

In [ ]:
constr_nodes, constr_edges, constr_score = construction_heuristic(nodes,edges,s)
print("number of plexes: "+ str(len(constr_nodes["splex"].unique())))
print("score: " +str(constr_score))
print(constr_edges)

# Randomized Construction heuristic

In [ ]:
rand_nodes, rand_edges, rand_score = randomized_greedy(nodes, edges, s, alpha = 0.75)
print("number of plexes: "+ str(len(rand_nodes["splex"].unique())))
print("score: " +str(rand_score))

# extract 1 node neighborhood

In [ ]:
extract_nodes, extract_edges, extract_score = extract1node(constr_nodes, constr_edges, constr_score, s, step = "best")
print("number of plexes: "+ str(len(extract_nodes["splex"].unique())))
print("score: " +str(extract_score))

# merge 2 plexes neighborhood

In [ ]:
merged_nodes, merged_edges, merged_score = merge2plexes(constr_nodes, constr_edges, constr_score, s, 
                                                        step = "best", add_neg_edges = True)
print("number of plexes: "+ str(len(merged_nodes["splex"].unique())))
print("score: " +str(merged_score))


In [ ]:
merged_nodes2, merged_edges2, merged_score2 = merge2plexes(constr_nodes, constr_edges, constr_score, s, 
                                                        step = "best", add_neg_edges = False)
print("number of plexes: "+ str(len(merged_nodes2["splex"].unique())))
print("score: " +str(merged_score2))

## do it again on the new best solution

In [ ]:
merged_nodes3, merged_edges3, merged_score3 = merge2plexes(merged_nodes, merged_edges, merged_score, s, step = "best")
print("number of plexes: "+ str(len(merged_nodes2["splex"].unique())))
print("score: " +str(merged_score2))

# einmal alles (mit scharf)

In [ ]:
instance_type = "inst_competition/"#"test_instances/" #"inst_competition/" #"inst_tuning/" # 
problem_instance = "heur051_n_300_m_20122" #"heur040_n_300_m_13358" #"heur001_n_10_m_31" 
path = "data/"+instance_type + problem_instance +".txt"
nodes, edges, s, n, m = create_problem_instance(path)
print(edges)
constr_nodes, constr_edges, constr_score = construction_heuristic(nodes, edges, s)
write_solution(constr_edges, problem_instance, algorithm = "construction")
print(constr_edges)

In [ ]:
def vnd(solution: Solution, neighborhoods: list[Neighborhood], max_minutes = 2) -> Solution:
    current_best: Solution = solution
    
    start = time.time()

    i = 0 # First neighborhood
    while i < len(neighborhoods):
        current_neighborhood: Neighborhood = neighborhoods[i]
        local_optimum = current_neighborhood.get_improvement(current_best, "first")

        print("i: " + str(i) + ", " + str(local_optimum.get_weight()))
        if not valid(local_optimum):
            break
            
        if not consistent(local_optimum):
            break

        if local_optimum.get_weight() < current_best.get_weight():
            current_best = local_optimum
            i = 0
        else:
            i += 1
        print((time.time()-start))
        if (time.time()-start) > (max_minutes * 60):
            break

    return current_best

In [ ]:
plex_merge_neighborhood = Neighborhood(lambda sol, step: merge2plexes(sol.get_nodes(), sol.get_edges(), sol.get_weight(), sol.get_s(), step, ))
add_edge_neighborhood = Neighborhood(lambda sol, step: add_edge_within_plex(sol.get_nodes(), sol.get_edges(), sol.get_weight(), sol.get_s(), step))
remove_edge_neighborhood = Neighborhood(lambda sol, step: remove_edge(sol.get_nodes(), sol.get_edges(), sol.get_weight(), sol.get_s(), step, skip_invalid=True))
extract_node_neighborhood = Neighborhood(lambda sol, step: extract1node(sol.get_nodes(), sol.get_edges(), sol.get_weight(), sol.get_s(), step))
move_node_neighborhood = Neighborhood(lambda sol, step: move1node(sol.get_nodes(), sol.get_edges(), sol.get_weight(), sol.get_s(), step))

In [ ]:
problem = Problem(path)
solution = grasp(problem, plex_merge_neighborhood, 20)
print(solution.get_weight())

In [ ]:
#problem = Problem("data/inst_competition/heur051_n_300_m_20122.txt")
#initial_solution = problem.get_heuristic_solution()
#print(valid(initial_solution))
#print(initial_solution.get_weight())
#print(initial_solution.get_nodes())
#write_solution(initial_solution.get_edges(), "heur051_n_300_m_20122", "construction")
#local_optimum = vnd(initial_solution, [add_edge_neighborhood, remove_edge_neighborhood, extract_node_neighborhood, plex_merge_neighborhood])
print(valid(local_optimum))
write_solution(initial_solution.get_edges(), "heur051_n_300_m_20122", "vnd")

In [ ]:
#plex_merge_neighborhood.get_improvement(inital_solution, ImprovementType.BEST)
#add_edge_neighborhood.get_improvement(inital_solution, ImprovementType.BEST)
#remove_edge_neighborhood.get_improvement(inital_solution, ImprovementType.BEST)
#extract_node_neighborhood.get_improvement(inital_solution, ImprovementType.BEST)

local_optimum.get_edges()

In [ ]:
instances = ["heur049_n_300_m_17695", "heur050_n_300_m_19207", "heur051_n_300_m_20122"]
problems = dict()
heuristics = dict()
randomized = dict()
vnds = dict()
for instance in instances:
    curr_problem = Problem("data/inst_competition/" + instance + ".txt")
    problems[instance] = curr_problem
    
    curr_heuristic = curr_problem.get_heuristic_solution()
    heuristics[instance] = curr_heuristic
    write_solution(curr_heuristic.get_edges(), instance, "construction")
    
    best_randomized = None
    for i in range(10):
        curr_randomized = curr_problem.get_randomized_solution()
        if (best_randomized == None) or (curr_randomized.get_weight() < best_randomized.get_weight()):
            best_randomized = curr_randomized
    randomized[instance] = best_randomized
    write_solution(curr_randomized.get_edges(), instance, "randomized")

    curr_vnd = vnd(curr_heuristic, [add_edge_neighborhood, remove_edge_neighborhood, extract_node_neighborhood, plex_merge_neighborhood], 120)
    vnds[instance] = curr_vnd
    write_solution(curr_vnd.get_edges(), instance, "vnd")

In [27]:
def grasp(randomized_solutions: list[Solution]):
    best_solution: Solution = None
    for solution in randomized_solutions:
        return vnd(solution, [add_edge_neighborhood, remove_edge_neighborhood, plex_merge_neighborhood], 15)
        if (best_solution == None) or (local_optimum.get_weight() < best_solution.get_weight()):
            best_solution = local_optimum
    return best_solution

grasp_solutions = dict()

for i in range(3):
    grasp_solution = grasp([randomized[instances[i]]])
    print("start")
    grasp_solutions[instances[i]] = grasp_solution
    print("end")
    print(grasp_solution.get_weight())

    file="output/" + instances[i] + "_grasp.txt"

    constr_edges = grasp_solution.get_edges()

    final_solution = constr_edges.loc[((constr_edges["e"]==0)&(constr_edges["w"]<=0))|
                                ((constr_edges["e"]==1)&(constr_edges["w"]>0)), ["n1", "n2"]]
    f = open(file, "w") 
    f.write(instances[i]+"\n")
    f.close()
    final_solution.to_csv(file, mode='a', index=False, header=False, sep = " ")

0.06 seconds
i: 0, 30667
0.7477710247039795
0.25 seconds
i: 1, 30662
1.6403324604034424
0.03 seconds
i: 0, 30663
2.312642812728882
0.32 seconds
i: 1, 30661
3.2957022190093994
0.03 seconds
i: 0, 30662
3.9870893955230713
0.34 seconds
i: 1, 30659
4.9910173416137695
0.03 seconds
i: 0, 30660
5.670255184173584
0.42 seconds
i: 1, 30658
6.760603189468384
0.03 seconds
i: 0, 30659
7.454958200454712
1.06 seconds
i: 1, 30657
9.164093494415283
0.03 seconds
i: 0, 30658
9.888686656951904
1.15 seconds
i: 1, 30652
11.699446439743042
0.03 seconds
i: 0, 30653
12.433135032653809
1.12 seconds
i: 1, 30649
14.215703964233398
0.04 seconds
i: 0, 30650
14.898224353790283
1.07 seconds
i: 1, 30645
16.611432790756226
0.03 seconds
i: 0, 30646
17.305001258850098
1.06 seconds
i: 1, 30644
18.98905038833618
0.03 seconds
i: 0, 30645
19.680826663970947
1.1 seconds
i: 1, 30643
21.429932594299316
0.03 seconds
i: 0, 30644
22.091285467147827
1.1 seconds
i: 1, 30642
23.849754571914673
0.03 seconds
i: 0, 30643
24.5152170658111